In [ ]:
import bs4
import requests
from lxml import html
import pandas as pd
from pprint import pprint
import numpy as np

# pd.set_option('display.max_columns', 10) 
# pd.set_option('display.max_rows', 500) 

In [ ]:
#get web data

#link for web data
url="https://www.citypopulation.de/en/ghana/admin/"

res=requests.get(url)
res.raise_for_status() #check if link is valid

<h1><b>BEAUTIFULSOUP</b></h1>


In [ ]:
#parse web data using bs4

data=bs4.BeautifulSoup(res.content, 'html.parser')

In [ ]:
# get table columns which should be marked with table head tag 'th'

head = data.find_all('th')

columns=[]

for td in head:
    columns.append(td.text)

columns=columns[:4]
pprint(columns)

In [ ]:
body = data.find_all('tbody')


body_rows=[]

for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
body_rows = body_rows[:len(body_rows)-4]
pprint(len(body_rows)

<h2><b>REGIONAL DATA</b></h2>


In [ ]:
regions = []
i = 0
while i < len(body_rows):
    regions.append(body_rows[i])
    i+=2

del regions[-1]

print(regions) 

In [ ]:
regcolumns=columns.copy()
regcolumns.append('Misc')

In [ ]:
df = pd.DataFrame(regions, columns=regcolumns)
df = df.drop(df[['Misc', 'Status']],axis=1)
df.columns = ['Region', 'Census(2010)', 'Census(2021)']
df.replace(',','',regex=True,inplace=True)
df[['Census(2010)', 'Census(2021)']]=df[['Census(2010)','Census(2021)']].astype('int64')
df

<h2><b>DISTRICTS DATA</b></h2>


In [ ]:
district_row = []

i = 1
while i < len(body_rows):
    district_row.append(body_rows[i])
    i+=2
del district_row[-1]
print(len(district_row))

In [ ]:
# district_row = []
# 
# i = 1
# while i < len(body_rows):
#     district_row.append(body_rows[i])
#     i+=2
# pprint(district_row)

In [ ]:
district = []
status = []
Census(2010) = []
Census(2021) = []

In [ ]:
for row in district_row:
    j=0
    while j < len(row):
        district.append(row[j])
        j+=5
pprint(len(district))

In [ ]:
for row in district_row:
    j=1
    while j < len(row):
        status.append(row[j])
        j+=5
pprint(len(status))

In [ ]:
for row in district_row:
    j=2
    while j < len(row):
        Census(2010).append(row[j])
        j+=5
pprint(len(Census(2010)))

In [ ]:
for row in district_row:
    j=3
    while j < len(row):
        Census(2021).append(row[j]) 
        j+=5
pprint(len(Census(2021)))

In [ ]:
#| error: false
#| warning: false


df=pd.DataFrame(data={'District':district, 'Status':status, 'Census(2010)':Census(2010), 'Census(2021)':Census(2021)})
df.replace(',','',regex=True,inplace=True)
df[['Census(2010)', 'Census(2021)']] = df[['Census(2010)', 'Census(2021)']].replace("...", np.NaN)
df[['Census(2010)', 'Census(2021)']]=df[['Census(2010)','Census(2021)']].astype('int64')
pprint(df)

<h2><b>NATIONAL</b></h2>


In [ ]:
country=data.find_all('tbody', class_="admin0")
td=country[0].find_all('td')

country=[]
for i in range(len(td)-1):
    if i>=2:
        country.append(td[i].text)
#country

In [ ]:
df=pd.DataFrame([country], columns=['Total Population (2010 Census(2010))', 'Census(2021) Population(2020)'])
df

In [ ]:
#scrape population distribution data for Ghana

details=data.find_all(class_='addinfo') 

In [ ]:
#add distribution table values to 

data1=[]
data2=[]

for i in range(len(details)):
    section=details[i].find_all('td')
    for j in range(len(section)):
        if j%2==0:
            data1.append(section[j].text)
        else:
            data2.append(section[j].text)

In [ ]:
pop_dist={}

#add the column headings and values to dictionary newdata
for col in data1:
    for pop in data2:
        pop_dist[col]=pop

In [ ]:
#create dataframe from dictionary

df = pd.DataFrame([pop_dist])
#df.reset_index(drop=True, inplace=True)


#remove , from values
df=df.replace(',', '',regex=True)


#convert all value to interger
df[list(df.columns)] = df[list(df.columns)].astype('int64')
df

In [ ]:
#df = df.append(newdata, ignore_index=True)
#df

In [ ]:
#create dataframe from dictionary with list values using Series

#columns=['Gender', 'Gender Pop', 'Age Group', 'Age Group Pop', 'Age Distribution', 'Age Distribution Pop' ]

#df2=pd.DataFrame({columns[0]:pd.Series(data1[0:2]), columns[1]:pd.Series(data2[0:2]), columns[2]:pd.Series(data1[2:5]),columns[3]:pd.Series(data2[2:5]), columns[4]:pd.Series(data1[5:]),columns[5]:pd.Series(data2[5:])})
#df2.fillna('')

In [ ]:
#create dataframe from dictionary with list values using from_list

#df3=pd.DataFrame.from_dict({columns[0]:data1[0:2], columns[1]:data2[0:2], columns[2]:data1[2:5], 
                            #columns[3]:data2[2:5], columns[4]:data1[5:],columns[5]:data2[5:]}, orient='index').T
#df3.fillna('')

<h1><b>LXML</b></h1>


In [ ]:
data=html.fromstring(res.content)

<h2><b>REGIONAL DATA</b></h2>


In [ ]:
#scrape using xpath(get this from inspecting the page and copying the full xpath
regions=[] 
cen_pop=[]
proj_pop=[]
area_size=[]

r_i=1
while r_i < 32:
    
    region=data.xpath(f'/html/body/article/table/tbody{[r_i]}/tr/td[1]/a/span/text()')
    Census(2010)=data.xpath(f'/html/body/article/table/tbody{[r_i]}/tr/td[3]/text()')
    Census(2021)_pop=data.xpath(f'/html/body/article/table/tbody{[r_i]}/tr/td[4]/text()')
    area=data.xpath(f'/html/body/article/table/tbody{[r_i]}/tr[2]/td/p[4]/span/text()')
    

    
    regions.append(region[0])
    cen_pop.append(Census(2010)[0])
    proj_pop.append(Census(2021)_pop[0])
    area_size.append(area)
    r_i+=2

pprint(regions)

In [ ]:
#create dataframe from data

reg_pop = pd.DataFrame({'Region':regions, 'Census(2010)':cen_pop, 'Census(2021)':proj_pop})
reg_pop=reg_pop.replace(',','', regex=True)
reg_pop[['Census(2010)', 'Census(2021)']]=reg_pop[['Census(2010)', 'Census(2021)']].astype('int64')
reg_pop

#reg_pop.to_csv('Population_by_Region_(GH).csv', index=False)

In [ ]:
#transform the data using melt so both populations can do a grouped graph

#newreg_pop = pd.melt(reg_pop, id_vars='Region', var_name='Type', value_name='Population')

<h2><b>DISTRICTS DATA</b></h2>


In [ ]:
districts=[]
dist_cen_pop=[]
dist_proj_pop=[]
status=[]

In [ ]:
districts=[]
dist_cen_pop=[]
dist_proj_pop=[]
status=[]

dbi=2
while dbi<33:
    
    district=data.xpath(f'/html/body/article/table/tbody{[dbi]}/tr/td/a/span/text()')
    Census(2010)=data.xpath(f'/html/body/article/table/tbody{[dbi]}/tr/td[3]/text()')
    proj_pop=data.xpath(f'/html/body/article/table/tbody{[dbi]}/tr/td[4]/text()')
    dist_type=data.xpath(f'/html/body/article/table/tbody{[dbi]}/tr/td[2]/text()')
    
    for dist in district:
        districts.append(dist)
    for popn1 in Census(2010):
        dist_cen_pop.append(popn1)
    for popn2 in proj_pop:
        dist_proj_pop.append(popn2)
    for stat in dist_type:
        status.append(stat)
    dbi+=2

In [ ]:
dist_pop=pd.DataFrame(data={'District':districts, 'Status':status, 'Census(2010)':dist_cen_pop, 'Census(2021)':dist_proj_pop})
dist_pop.replace(',','',regex=True,inplace=True)
dist_pop[['Census(2010)', 'Census(2021)']]=dist_pop[['Census(2010)','Census(2021)']].astype('int64')
dist_pop
#dist_pop.to_csv('Population_by_District_(GH).csv', index=False)

In [ ]:
country=data.xpath('/html/body/article/table/tbody[33]/tr/td[1]/text()')
Census(2010)=data.xpath('/html/body/article/table/tbody[33]/tr/td[3]/text()')
Census(2021)=data.xpath('/html/body/article/table/tbody[33]/tr/td[4]/text()')